In [1]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import numpy as np
import nltk
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords

from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from textblob import TextBlob

In [2]:
df = pd.read_excel('War Sentiment Analysis.xlsx')

In [3]:
df.head()

,user_id,tweet,language
0,3031528348,@merjaya I urge you to stop contributing into ...,en
1,1225800000000000000,Russia to ban sunflower seed exports and impos...,en
2,1394360000000000000,@nathaliejacoby1 On Russia-Trump Saga in 2016 ...,en
3,982157000000000000,It turns out that people living on islands &am...,en
4,199448306,Russia Attacks Ukraine - LIVE BREAKING NEWS CO...,en


In [35]:
df.shape

(10001, 4)

In [37]:
df1 = df[df['language']=='en']
df1.shape

(8483, 4)

In [4]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def pre_process(text):
    text = str(text).lower()
    text = re.sub(r"[-()\"#/@;:{}`+=~|.!?,'0-9]", "", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stop]
    text=" ".join(text)
    return text

In [38]:
df1['clean_tweets'] = df1['tweet'].apply(lambda x: pre_process(x))

<ipython-input-38-bf6cdf76df9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['clean_tweets'] = df1['tweet'].apply(lambda x: pre_process(x))


In [51]:
df1.head()

,user_id,tweet,language,clean_tweets
0,3031528348,@merjaya I urge you to stop contributing into ...,en,merjaya urge stop contributing deaths innocent...
1,1225800000000000000,Russia to ban sunflower seed exports and impos...,en,russia ban sunflower seed exports impose quota...
2,1394360000000000000,@nathaliejacoby1 On Russia-Trump Saga in 2016 ...,en,nathaliejacoby russiatrump saga us election o...
3,982157000000000000,It turns out that people living on islands &am...,en,turns people living islands amp arent white sa...
4,199448306,Russia Attacks Ukraine - LIVE BREAKING NEWS CO...,en,russia attacks ukraine live breaking news cov...


In [40]:
Tweets = df1['clean_tweets'].values

In [41]:
Tweets

array(['merjaya urge stop contributing deaths innocents ukraine every ruble falls taxes russia turns tears death ukrainian children stop traffic goods transport fromto russian federation show support actions  httpstcogjtdltvl',
       'russia ban sunflower seed exports impose quota sunoil   httpstcooyndajbfi  httpstcofyfnobjq',
       'nathaliejacoby russiatrump saga  us election one may say â€œlusionâ€\x9d â€œlusionâ€\x9d amp doesnt matter whether â€œallusionâ€\x9d â€œcollusionâ€\x9d â€œexclusionâ€\x9dor â€œinclusionâ€\x9d â€œdelusionâ€\x9d â€œcrimeâ€\x9d â€œcrimeâ€\x9d regardless fact criminal â€œwhite collarâ€\x9d â€œnonwhite collarâ€\x9d',
       ...,
       'kajakallas bild julianroepcke noble know aint doable lets focus  freezing sort assets linked ru state  working stable solution cut acquisitions energy resources russia',
       'kellyesorelle orionmidas yep like believe george papadopoulos getting caught likely flynn talking fbi without lawyer set trump russia hoax',
       'j

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
newsVec = [nltk.word_tokenize(word) for word in Tweets]
newsVec

[['merjaya',
  'urge',
  'stop',
  'contributing',
  'deaths',
  'innocents',
  'ukraine',
  'every',
  'ruble',
  'falls',
  'taxes',
  'russia',
  'turns',
  'tears',
  'death',
  'ukrainian',
  'children',
  'stop',
  'traffic',
  'goods',
  'transport',
  'fromto',
  'russian',
  'federation',
  'show',
  'support',
  'actions',
  'httpstcogjtdltvl'],
 ['russia',
  'ban',
  'sunflower',
  'seed',
  'exports',
  'impose',
  'quota',
  'sunoil',
  'httpstcooyndajbfi',
  'httpstcofyfnobjq'],
 ['nathaliejacoby',
  'russiatrump',
  'saga',
  'us',
  'election',
  'one',
  'may',
  'say',
  'â€œlusionâ€\x9d',
  'â€œlusionâ€\x9d',
  'amp',
  'doesnt',
  'matter',
  'whether',
  'â€œallusionâ€\x9d',
  'â€œcollusionâ€\x9d',
  'â€œexclusionâ€\x9dor',
  'â€œinclusionâ€\x9d',
  'â€œdelusionâ€\x9d',
  'â€œcrimeâ€\x9d',
  'â€œcrimeâ€\x9d',
  'regardless',
  'fact',
  'criminal',
  'â€œwhite',
  'collarâ€\x9d',
  'â€œnonwhite',
  'collarâ€\x9d'],
 ['turns',
  'people',
  'living',
  'islands',
  

In [52]:
model = Word2Vec(newsVec, min_count=10, vector_size=64, window=4, sg=1, epochs=20)

In [53]:
model.wv.get_vector('russia')

array([-2.2436379e-01, -7.1515262e-02,  4.6049726e-01,  5.8950251e-01,
        1.1245269e-04, -3.9315861e-01,  1.6032106e-01,  4.7827452e-02,
       -4.2453220e-01, -4.7524415e-02,  1.6639183e-01, -3.2160801e-01,
        5.1223791e-01, -3.8578011e-02,  1.0779093e-01, -1.4861648e-01,
       -1.4594948e-01,  2.3324184e-01, -3.3214718e-01,  1.7905933e-01,
        3.1285638e-01,  3.1567466e-01,  3.7688583e-01, -2.3442714e-01,
        6.3766792e-02,  7.1532063e-02, -9.4514325e-02, -1.2588862e-01,
        8.0522381e-02, -1.3499483e-01, -3.6652457e-02,  6.9268614e-02,
        2.5227487e-01, -1.2514064e-04, -9.0797953e-02,  2.1153665e-01,
        1.7021725e-01, -1.2636110e-02,  3.0312347e-01, -1.2976498e-02,
       -3.2639143e-01, -9.7842470e-02, -6.1766696e-01,  2.7746451e-01,
        1.8087196e-01,  1.8812072e-02, -2.1835993e-01, -7.5079940e-02,
       -2.0691438e-01,  1.4539978e-01, -3.3619288e-01, -1.1440404e-01,
       -6.9300339e-02,  1.5978017e-01,  9.4987534e-02,  9.1525592e-02,
      

In [54]:
model.wv['russia']

array([-2.2436379e-01, -7.1515262e-02,  4.6049726e-01,  5.8950251e-01,
        1.1245269e-04, -3.9315861e-01,  1.6032106e-01,  4.7827452e-02,
       -4.2453220e-01, -4.7524415e-02,  1.6639183e-01, -3.2160801e-01,
        5.1223791e-01, -3.8578011e-02,  1.0779093e-01, -1.4861648e-01,
       -1.4594948e-01,  2.3324184e-01, -3.3214718e-01,  1.7905933e-01,
        3.1285638e-01,  3.1567466e-01,  3.7688583e-01, -2.3442714e-01,
        6.3766792e-02,  7.1532063e-02, -9.4514325e-02, -1.2588862e-01,
        8.0522381e-02, -1.3499483e-01, -3.6652457e-02,  6.9268614e-02,
        2.5227487e-01, -1.2514064e-04, -9.0797953e-02,  2.1153665e-01,
        1.7021725e-01, -1.2636110e-02,  3.0312347e-01, -1.2976498e-02,
       -3.2639143e-01, -9.7842470e-02, -6.1766696e-01,  2.7746451e-01,
        1.8087196e-01,  1.8812072e-02, -2.1835993e-01, -7.5079940e-02,
       -2.0691438e-01,  1.4539978e-01, -3.3619288e-01, -1.1440404e-01,
       -6.9300339e-02,  1.5978017e-01,  9.4987534e-02,  9.1525592e-02,
      

In [55]:
model.wv['nato']

array([-0.03238737, -0.09948107, -0.5930821 ,  1.0407748 , -0.112448  ,
       -0.6364069 ,  0.37744886, -0.48979467,  0.04568773, -0.09015251,
        0.6641579 , -0.04314345, -0.38515276,  0.6622564 ,  0.06974178,
        0.15272579, -0.9205747 , -0.23967329, -0.20321201,  0.6182479 ,
        0.59980327,  0.18993203,  0.7398867 ,  0.16426405,  0.21396653,
        0.2246325 , -0.05297641, -0.59491986,  0.46034664, -0.48546076,
       -0.508369  ,  0.61534077,  0.07856289, -0.42037788, -0.39569703,
       -0.33746448,  0.09092992,  0.2954209 ,  0.052495  ,  0.540744  ,
       -0.26687697, -0.01298158, -0.00580572, -0.66138816,  0.13558161,
        0.262408  , -0.16187656,  0.12859702, -0.00844191,  0.21676011,
        0.17455402,  0.6871373 ,  0.6868122 ,  0.7451236 , -0.21355976,
        0.32042348,  0.1491277 , -0.6104506 , -0.27468306, -0.07393938,
       -0.29451942, -0.3792243 , -0.10648165, -0.1436678 ], dtype=float32)

In [56]:
model.wv['people']

array([ 0.11533378, -0.35457468,  0.29404423,  0.23199023,  0.31994817,
       -0.1079428 ,  0.47180367, -0.4049011 , -0.773204  , -0.07161908,
       -0.02512707, -0.628449  ,  0.22174497, -0.33609885,  0.4639082 ,
        0.22968823,  0.03530172, -0.01109522,  0.04373783,  0.7035648 ,
        0.2183944 ,  0.5207613 , -0.06829771,  0.13347879, -0.5669525 ,
        0.17759947,  0.16042872,  0.2586072 ,  0.05469624, -0.3316796 ,
        0.13758999,  0.02542576,  0.18002658, -0.37813815,  0.41142607,
        0.08146244, -0.04890215, -0.41518945,  0.7234861 ,  0.361046  ,
       -0.13851662, -0.10618343, -0.32504213, -0.05505055,  0.23305558,
       -0.53865564,  0.4210913 , -0.32413912,  0.3913001 ,  0.6555197 ,
        0.26239806,  0.00656752,  0.01344915,  0.43215197,  0.05953129,
       -0.5469536 , -0.44999704, -0.33730453,  0.06688982, -0.62640053,
       -0.5616543 , -0.2919351 , -0.07311738, -0.232734  ], dtype=float32)

In [57]:
model.wv.most_similar('russia')

[('putin', 0.5868346691131592),
 ('ukraine', 0.5437271595001221),
 ('liberating', 0.5352844595909119),
 ('light', 0.5323747396469116),
 ('ratio', 0.5314297676086426),
 ('basically', 0.5167142152786255),
 ('consider', 0.5048967003822327),
 ('dark', 0.5001432299613953),
 ('thehill', 0.4925759732723236),
 ('starts', 0.4923005700111389)]

In [62]:
vec = model.wv['wealth']+model.wv['billion']
model.wv.most_similar([vec])

[('billion', 0.9308596849441528),
 ('wealth', 0.7588446140289307),
 ('usd', 0.699654221534729),
 ('half', 0.6970170736312866),
 ('feed', 0.6699566841125488),
 ('frozen', 0.6529076099395752),
 ('deitaone', 0.6377846002578735),
 ('cash', 0.631432056427002),
 ('backing', 0.6255795359611511),
 ('pegged', 0.625181257724762)]

In [65]:
model.wv.most_similar('market')

[('stock', 0.684349775314331),
 ('increasing', 0.666532576084137),
 ('discount', 0.6624168157577515),
 ('stocks', 0.6539741158485413),
 ('products', 0.6457788348197937),
 ('imports', 0.6402027010917664),
 ('risks', 0.6326966285705566),
 ('deitaone', 0.6316362023353577),
 ('shortage', 0.6170698404312134),
 ('sanctioning', 0.6072800755500793)]

In [75]:
model.predict_output_word('wealth')

[('w', 0.007040947),
 ('iran', 0.0038144914),
 ('india', 0.0028424968),
 ('amp', 0.0028164012),
 ('behind', 0.0027976993),
 ('state', 0.0026869436),
 ('politics', 0.0024055457),
 ('via', 0.0023298417),
 ('things', 0.0023210172),
 ('democracy', 0.0023084725)]

## Word2vec from pre-trained models

In [66]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import numpy as np
import nltk

In [67]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [68]:
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [69]:
glove_vectors.most_similar('twitter')

[('Twitter', 0.8908904194831848),
 ('Twitter.com', 0.7536780834197998),
 ('tweet', 0.7431626319885254),
 ('tweeting', 0.7161932587623596),
 ('tweeted', 0.7137226462364197),
 ('facebook', 0.6988551616668701),
 ('tweets', 0.6974530816078186),
 ('Tweeted', 0.6950210928916931),
 ('Tweet', 0.6875007152557373),
 ('Tweeting', 0.6845167279243469)]

In [70]:
vec = glove_vectors['Germany']-glove_vectors['Berlin']+glove_vectors['Paris']
glove_vectors.most_similar([vec])

[('France', 0.7724406123161316),
 ('Paris', 0.6798243522644043),
 ('Belgium', 0.598486065864563),
 ('Germany', 0.5652832388877869),
 ('extradites_Noriega', 0.5623601078987122),
 ('Villebon_Sur_Yvette', 0.5570637583732605),
 ('Spain', 0.550815761089325),
 ('Italy', 0.5462924838066101),
 ('Marseille', 0.5372346639633179),
 ('Switzerland', 0.5364957451820374)]

In [71]:
vec = glove_vectors['Germany']-glove_vectors['Berlin']+glove_vectors['Delhi']
glove_vectors.most_similar([vec])

[('Delhi', 0.7607876062393188),
 ('India', 0.7546477913856506),
 ('Haryana', 0.6704527735710144),
 ('Kolkata', 0.6515370607376099),
 ('Hyderabad', 0.6513314247131348),
 ('Uttar_Pradesh', 0.6438483595848083),
 ('NEW_DELHI', 0.6347569227218628),
 ('Andhra_Pradesh', 0.6346529126167297),
 ('Chennai', 0.6248441934585571),
 ('Chandigarh', 0.6227700710296631)]

In [72]:
vec = glove_vectors['man']+glove_vectors['woman']
glove_vectors.most_similar([vec])

[('woman', 0.948119044303894),
 ('man', 0.9308565258979797),
 ('teenage_girl', 0.721599280834198),
 ('girl', 0.7193813323974609),
 ('teenager', 0.6853229403495789),
 ('boy', 0.6776923537254333),
 ('teen_ager', 0.5814452767372131),
 ('lady', 0.5798472762107849),
 ('teenaged_girl', 0.5797287225723267),
 ('person', 0.5755369663238525)]